In [2]:
import math
import numpy as np
import pandas as pd
import csv
from tqdm import tqdm

print('环境创建成功')

环境创建成功


In [2]:
class dotPair(object):  # 点对，用于表示组成三角形的三个线段
    abc = []
    def __init__(self, sPoint=None, ePoint=None):
        self.sPoint = sPoint
        self.ePoint = ePoint
        self.sp = [self.sPoint[0], self.sPoint[1]]
        self.ep = [self.ePoint[0], self.ePoint[1]]
        self.abc = [self.sp[1] - self.ep[1], self.ep[0] - self.sp[0], np.linalg.det([self.sp, self.ep])]

In [3]:
class line(object):  # 射线类，具有起始点以及方向，用于表示每个三角形上水流的起始点以及流动的方向
    sPoint = []
    theta = []
    alpha = []
    abc = []

    def __init__(self, sPoint=None, alpha=None):  # 输入的alpha为坡向
        self.sPoint = sPoint
        if alpha[0] <= math.pi / 2:   
            theta = math.pi / 2 - alpha[0]      #此处的theta为直线的倾斜角
        elif math.pi / 2 < alpha[0] <= math.pi * 3 / 2:
            theta = math.pi * 3 / 2 - alpha[0]
        else:
            theta = math.pi * 5 / 2 - alpha[0]
        self.theta = [round(np.cos(theta), 6), round(np.sin(theta), 6)]   # 计算倾斜角的余弦正弦值，用于求射线的一般式方程的abc系数
        self.alpha = [round(np.sin(alpha[0]), 6), round(np.cos(alpha[0]), 6)]  # 计算坡向的正弦余弦值，用于判别交点是否在射线方向上
        self.sp = [self.sPoint[0], self.sPoint[1]]
        self.abc = [self.theta[1], -self.theta[0], np.linalg.det([self.theta, self.sp])]

In [4]:
def ifIntersect(line1, line2):  # 判断射线与线段是否有交点，如果有交点则返回1。
    sFlag = np.sum(np.array(line1.abc) * np.array([line2.sPoint[0], line2.sPoint[1], 1]))
    eFlag = np.sum(np.array(line1.abc) * np.array([line2.ePoint[0], line2.ePoint[1], 1]))
    if round(sFlag * eFlag) <= 0:   # 有交点的话，将线段的两个端点带入射线的方程ax+by+c，得出的结果一定是异号。
        num = 1
    else:
        num = 0
    return num

In [5]:
def getDistance(ray, dp):  # l1是射线，l2是线段
#     Dotabc = [dp.sPoint[1] - dp.ePoint[1], dp.ePoint[0] - dp.sPoint[0], np.linalg.det([dp.sp, dp.ep])]  # 计算线段的一般式方程的abc三个系数
    delta = ray.abc[0] * dp.abc[1] - dp.abc[0] * ray.abc[1]
    if delta == 0:
        return None
    c1 = (ray.abc[1] * dp.abc[2] - dp.abc[1] * ray.abc[2]) / delta
    c2 = (dp.abc[0] * ray.abc[2] - ray.abc[0] * dp.abc[2]) / delta
    cross = [round(c1, 2), round(c2, 2)]  # 利用判别式求射线与线段的交点坐标
    distance = math.sqrt((ray.sPoint[0] - cross[0]) ** 2 + (ray.sPoint[1] - cross[1]) ** 2)
    if cross == ray.sp or distance < 0.005:   # 如果交点恰好是射线的起点，则返回None，否则后续迭代计算时，若第一步的线段恰好是上一步交点所在的线段，判断会停止
        return None
    d1 = (cross[0] - ray.sPoint[0]) * ray.alpha[0]   # 判断交点是否在射线方向上。若在射线方向上，则交点和始点的x、y的差值分别与坡向的正弦、余弦三角函数同号
    d2 = (cross[1] - ray.sPoint[1]) * ray.alpha[1]
    d3 = ifIntersect(ray, dp)
    if d1 >= 0 and d2 >= 0 and d3 == 1:
#         distance = math.sqrt((ray.sPoint[0] - cross[0]) ** 2 + (ray.sPoint[1] - cross[1]) ** 2)
        return cross, round(distance, 6)

In [ ]:
tn = pd.read_csv("/300mspacejoin.csv")
for i in tn.index:
    if tn.iloc[i,1] < 180:
        tn.iloc[i,1] = (tn.iloc[i,1] + 180)*math.pi/180
    else:
        tn.iloc[i,1] = (tn.iloc[i,1] - 180)*math.pi/180

# 数据清洗

In [ ]:
tin_nodes = tn.groupby(['Tri_Index','Slope_Pct','Aspect','Node_Index']).first()

In [ ]:
temp = tin_nodes.to_dict()['TinNod.xyz']
tndict = {}

## 字典tndict（Tin-Node Dict)

格式如下：

```python
tndict = {
    '(tri_index_1, slope_Deg, aspect)': 
        {
        'node_index_1': 'coord_x, coord_y，coord_z',
        'node_index_2': 'coord_x, coord_y，coord_z',
        'node_index_3': 'coord_x, coord_y，coord_z'
        },
    ...,
    '(tri_index_n, slope_Deg, aspect)': 
        {
        'node_index_1': 'coord_x, coord_y，coord_z',
        'node_index_2': 'coord_x, coord_y，coord_z',
        'node_index_3': 'coord_x, coord_y，coord_z'
        }
}
```
该字典中，键为一个元组，元组中的元素为三角形编号，该三角形的坡度，坡向；值为一个字典,该字典有三个键值对，键为组成对应值字典的键中三角形的三个节点编号，值为该节点编号的xyz坐标。

In [ ]:
for k in temp:
    tid = (k[0],k[1],k[2])
    nid = k[3]
    nodxzy = temp[k]
    if tid not in tndict:
        tndict[tid] = {}
    tndict[tid][nid] = nodxzy   

In [13]:
nodes_tin =tn.groupby(['Node_Index','Tri_Index'])['Slope_Pct','Aspect'].first()
#nodes_tin.index

C:\Users\LZY\AppData\Local\Temp/ipykernel_1960/4093753947.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  nodes_tin =tn.groupby(['Node_Index','Tri_Index'])['Slope_Pct','Aspect'].first()


## 字典ntdict（Node - Tin Dict）

用于查询某个节点属于哪些三角形的组成节点。

In [ ]:
ntdict = {}
for i in nodes_tin.index:
    nid = i[0]
    tid = i[1]
    if nid not in ntdict:
        ntdict[nid] = []
    ntdict[nid].append(tid)

## 字典pt（每个三角面片上随机采样的起始点）

格式如下：

```python
pt = {
    '(tri_index_1, slope_Deg, aspect)': 'coord_x, coord_y',
    ...,
    '(tri_index_n, slope_Deg, aspect)': 'coord_x, coord_y'
}
```
该字典中，键为一个元组，元组中的元素为三角形编号，该三角形的坡度，坡向，代表该随机点所在的三角面片的信息；值是随机点的xy坐标

In [ ]:
points = pd.read_csv('/300mcenterpoint.csv')

In [ ]:
for i in points.index:
    if points.iloc[i,1] < 180:
        points.iloc[i,1] = (points.iloc[i,1] + 180)*math.pi/180
    else:
        points.iloc[i,1] = (points.iloc[i,1] - 180)*math.pi/180
pt = points.groupby(['Tri_Index','Slope_Pct','Aspect']).first()
pt = pt.to_dict()['p.xy']

In [17]:
trilist = []
for key in pt:
    trilist.append(key)
# trilist[6]  #随机点所在的三角形的编号，坡度，坡向（反向

## 字典tridict

通过tirdict来使用三角形的索引编号来定位三角形

In [ ]:
tridict = {}
for i in trilist:
    key = i[0]
    value = i
    tridict[key] = value

In [22]:
def point(p):  # 字符串坐标转化为列表形式的浮点坐标的函数
    num = p.count(',')
    if num < 2:   # 当坐标为输入坐标的'x, y, z'，即坐标中包含高程值z时，逗号","的数量为2，当输入的坐标没有z时则逗号的数量为1
        return [float(p.split(',')[0]), float(p.split(',')[1])]
    else:
        return [float(p.split(',')[0]), float(p.split(',')[1]), float(p.split(',')[2])]
# point(pt[tridict[6]])

In [23]:
# 水流抵达公共边时找到水流流入的下一个三角形的函数
def findtri(l1, l2, a):   # l1、l2是公共边两个顶点的所属三角形的列表，a是水流抵达公共边前所在的三角形编号
    i = [item for item in l1 if item in l2]   # 取两个列表的交集，得到共边的两个三角形的编号
    i.remove(a)  # 去掉原先所在的三角形编号，剩下的即为下一个要流入的三角形，若没有下一个三角形则说明水流抵达了边界
    if i ==[]:
        return None
    else:
        return i[0]

In [24]:
def getsum(tri_index):   # 计算编号为tri_index的三角形的三个节点的高程总和
    ttt = tndict[tridict[tri_index]]
    thlist = []
    for value in ttt.values():
        h = point(value)
        thlist.append(h[2])
    return sum(thlist)
# getsum(6)

In [25]:
# 当水流来到三角面片某一节点上，且该节点非局部最高点时，则水流流入三个节点高程总和较高的那一个三角面。
def gettri(n1, n2):   # 输入该节点编号和其周围最高的节点的编号
    l1 = ntdict[n1]
    l2 = ntdict[n2]
    tlist = [item for item in l1 if item in l2]    # 两点确定一条边，确定两个三角面片
    if len(tlist) == 1:
        return tlist[0]
    elif len(tlist) == 2:
        if getsum(tlist[0]) > getsum(tlist[1]):
            return tlist[0]
        else:
            return tlist[1]

In [26]:
def iftop(n_index):   # 当水流来到三角面片的某一节点上时，判断该节点是否为局部最高点
    tri = ntdict[n_index]
    pheight = []
    for i in tri:
        height = tndict[tridict[i]]
        pheight.append(height)
    nid = []
    for i in pheight:
        for key in i.keys():
            if key not in nid:
                nid.append(key)
    xyzlist = []
    for d in pheight:
        for i in d.values():
            pi = point(i)
            if pi not in xyzlist:
                xyzlist.append(pi)
    hlist = []
    for i in xyzlist:
        hlist.append(i[2])
    nh = dict(zip(nid, hlist))
    maxp = []    # 记录了该节点周围高程最高的节点的编号的列表
    for key, value in nh.items():
        if value == max(nh.values()):
            maxp.append(key)
    if n_index in maxp:     # 若该节点在具有最高高程的节点的编号列表里，则该节点视为局部最高点
        return None
    elif len(maxp) == 2:    # 若具有最高高程的节点的编号有两个（不包括输入节点本身），则三点确定一个三角面片。
        l1 = ntdict[maxp[0]]
        l2 = ntdict[maxp[1]]
        tri_index = [item for item in tri if item in l2 and item in l1]
        if tri_index:
            return tri_index[0]
        else:
            return 0.1
    elif len(maxp) ==1:
        tri_index = gettri(n_index, maxp[0])
        return tri_index    # 不是局部最高点则使用gettri函数获取下一个三角面片的编号。

In [27]:
# 判断三点是否一线的函数：ac + bc = ab，则c在ab上，并且返回高程c与高程较高的端点间的距离
def ifinline(a, b, c):   # 输入的a，b，c为[x, y, z]坐标
    ab = round(math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2), 2)
    ac = round(math.sqrt((a[0] - c[0]) ** 2 + (a[1] - c[1]) ** 2), 2)
    bc = round(math.sqrt((b[0] - c[0]) ** 2 + (b[1] - c[1]) ** 2), 2)
    ac_cb = round(ac + bc,2)
    if ac_cb == ab or abs(ac_cb - ab) < 0.04:
        if a[2] > b[2]:
            return [ac, a[2]]
        elif b[2] > a[2]:
            return [bc, b[2]]
        else:
            return None    # 两端点高程相同时返回0
    else:
        return None

In [28]:
# 当水流处于山脊时，水流去往高程较高的一端。
def ridge(p1, p2, p3, p):   # 输入三角形的三个节点
    p1p2p = ifinline(p1, p2 ,p)    # 判断点p在那条边上（三点一线）
    p1p3p = ifinline(p1, p3 ,p)
    p2p3p = ifinline(p2, p3, p)
    rrlist = []
    if p1p2p:
        if p1p2p[0] == 0:  
            return None    # 两端点高高程相等时返回None
        else:
            if p1[2] > p2[2]:
                rrlist.append([p1p2p, p1[:-1]])
            else:
                rrlist.append([p1p2p, p2[:-1]])
    if p1p3p:
        if p1p3p[0] ==0:
            return None
        else:
            if p1[2] > p3[2]:
                rrlist.append([p1p3p, p1[:-1]])
            else:
                rrlist.append([p1p3p, p3[:-1]])
    if p2p3p:
        if p2p3p[0] ==0:
            return None
        else:
            if p2[2] > p3[2]:
                rrlist.append([p2p3p, p2[:-1]])
            else:
                rrlist.append([p2p3p, p3[:-1]])
    if not rrlist:
        return None
    elif len(rrlist) == 2:
        if rrlist[0][0][1] > rrlist[1][0][1]:     #
            length = rrlist[0][0][0]
            nextp = rrlist[0][1]
        else:
            length = rrlist[1][0][0]
            nextp = rrlist[1][1]
    else:
        length = rrlist[0][0][0]
        nextp = rrlist[0][1]
    rr = [length, nextp]   # 返回起始点和山脊上较高的端点的两点间距作为单元坡长，返回下一水流起始点
    return rr

In [29]:
# 坡长递归计算函数
def getslope(p, tri_index):    # 输入水流路径生成起始点和其所在的三角形的编号
    nxy = []      # 顺序记录该三角面片的三个节点的坐标[x, y, z]的列表
    for i in tndict[tridict[tri_index]].values():
        nxy.append(point(i))
    p1 = nxy[0]
    p2 = nxy[1]
    p3 = nxy[2]

    pnode = []    # 顺序记录该三角面片的三个节点的不含高程的坐标[x, y]的列表
    for i in nxy:
        pnode.append(i[:-1])

    nindex = []   # 顺序记录该三角面片的三个节点的编号的列表
    for i in tndict[tridict[tri_index]].keys():
        nindex.append(i)
    n1 = nindex[0]
    n2 = nindex[1]
    n3 = nindex[2]

    if p in pnode:    # 若水流路径生成的起始点在pnode列表里，说明其在三角面片的节点上
        if p == pnode[0]:
            n_index = n1    # 获取节点的编号
        elif p == pnode[1]:
            n_index = n2
        else:
            n_index = n3
        triindex = iftop(n_index)    # 判断该节点是否是局部最高点，若不是则获取并更新下一个三角形的编号，若是则停止递归
        if not triindex:
            return 0
        elif triindex == 0.1:
            tri_indexri = triindex
        else:
            tri_index = triindex   # 更新三角形编号后，重新录入三个节点的坐标和编号。
            nxy = []
            for i in tndict[tridict[tri_index]].values():
                nxy.append(point(i))
            p1 = nxy[0]
            p2 = nxy[1]
            p3 = nxy[2]

            nindex = []
            for i in tndict[tridict[tri_index]].keys():
                nindex.append(i)
            n1 = nindex[0]
            n2 = nindex[1]
            n3 = nindex[2]

    route.append(tri_index)   # 记录水流流经的三角形的编号
    ray = line(sPoint=p, alpha=[tridict[tri_index][2]])   # 输入起始点p和三角形的坡向生成水流射线
    l1 = dotPair(sPoint=p1, ePoint=p2)    # 三角形的三个节点分别形成三条线段
    l2 = dotPair(sPoint=p1, ePoint=p3)
    l3 = dotPair(sPoint=p2, ePoint=p3)
    if getDistance(ray, l1) is not None:
        length = getDistance(ray, l1)[1]
        p = getDistance(ray, l1)[0]
        tri_index = findtri(ntdict[n1], ntdict[n2], tri_index)
    elif getDistance(ray, l2) is not None:
        length = getDistance(ray, l2)[1]
        p = getDistance(ray, l2)[0]
        tri_index = findtri(ntdict[n1], ntdict[n3], tri_index)
    elif getDistance(ray, l3) is not None:    # 三个if顺序判断水流射线与三条线段是否有交点，若有交点则计算单元坡长和交点坐标。
        length = getDistance(ray, l3)[1]
        p = getDistance(ray, l3)[0]    # 将交点作为下一个水流射线的起始点
        tri_index = findtri(ntdict[n2], ntdict[n3], tri_index)    # 水流抵达公共边时找到水流流入的下一个三角形
    else:     # 若水流射线与三条边都没有交点，说明该边为山脊
        pinridge = ridge(p1, p2, p3, p)
        if not pinridge:
            return 0
        else:
            length = pinridge[0]
            p = pinridge[1]
    routep.append(p)
    if tri_index is None:
        return 0
    else:
        slopel = length + getslope(p, tri_index)   # 坡长累加
    return slopel

In [ ]:
with open('/300mslopelength.csv', 'w', encoding='UTF8', newline='') as f:
    header = ['tri_id', 'slopelength', 'routep']
    writer = csv.writer(f)
    writer.writerow(header)
    for i in tqdm(tridict):
        p = point(pt[tridict[i]])
        routep = [p]
        route = []
        result = getslope(p, i)
        data = [i, result, routep]
        writer.writerow(data)